In [1]:
import pandas as pd
import numpy as np
import json
import math

## Load New Name

In [2]:
new_names = pd.read_csv("codeAndNamev2.csv")
new_names.head()

,New article name,Article,Description
0,00GovStr,00TRDBDKSMJ,"Redudant Governance Structures - Rowley, T; Be..."
1,00LatCon,00ELDKQQ,Spreading and shifting costs of lateral contro...
2,00WrkGrp,00PJHKMCDKODHDP,"Choosing Work Group Members; Hinds, PJ; Carley..."
3,01PreTran,01PDDKJMS,Pre-transitive balance mechanisms for signed n...
4,02Festsch,02NEFDKSN,Festschrift for Linton C Freeman: Introduction...


In [3]:
newInfo = list(zip(new_names['New article name'],new_names['Description']))
oldID = list(new_names['Article'])

# dictionary that look up the old article id for the new id and description
article_dict = dict(zip(new_names['Article'],new_names['New article name']))

# dictionary to look up the discription with new artical name
descrptn_dict = dict(zip(new_names['New article name'],new_names['Description']))

# Change data into one mode and two mode

## Load data

In [4]:
df = pd.read_excel("Krackhardt Web of Science Data v3.xlsx")
df = df.sample(n = 1000, random_state=0)
df.head()

,Article,Description,Authors,Author Full Name,Document Title,Publication Name,Publication year
1644,90DKASQ,NaN,"Rank, ON; Robins, GL; Pattison, PE","Rank, Olaf N.; Robins, Garry L.; Pattison, Phi...",Structural Logic of Intraorganizational Networks,ORGANIZATION SCIENCE,2010
5651,96KMCDKSN,NaN,"Aalbers, HL; Dolfsma, W","Aalbers, Hendrik Leendert; Dolfsma, Wilfred",Bridging firm-internal boundaries for innovati...,JOURNAL OF ENGINEERING AND TECHNOLOGY MANAGEMENT,2015
4818,10MTDKAMJ,NaN,"Sutanto, J; Kankanhalli, A; Tan, BCY","Sutanto, Juliana; Kankanhalli, Atreyi; Tan, Be...",Uncovering the relationship between OSS user s...,DECISION SUPPORT SYSTEMS,2014
4769,10MTDKAMJ,NaN,"Brennecke, J; Rank, O","Brennecke, Julia; Rank, Olaf",The firm's knowledge network and the transfer ...,RESEARCH POLICY,2017
2190,88DKSRNSPQ,NaN,"Sommer, A; Pearson, CM","Sommer, Amy; Pearson, Christine M.",Antecedents of creative decision making in org...,TECHNOLOGICAL FORECASTING AND SOCIAL CHANGE,2007


## Abnormal data detection

In [5]:
#Multi-records of the same Article-citation pair

article_citation = df.drop("Description", axis = 1).groupby(["Article", "Document Title"]).count()

multi_citation = article_citation[(article_citation["Authors"] > 1) 
                                 | (article_citation["Author Full Name"] > 1)
                                 | (article_citation["Publication Name"] > 1)
                                 | (article_citation["Publication year"] > 1)]

        
multi_citation = multi_citation.index.values.tolist()
multi_citation

[]

In [6]:
#Multi-description of the same article

article_description = df.dropna(subset=["Description"])[["Article", "Description"]].groupby(["Article"]).count()

multi_description = article_description[article_description["Description"] > 1]
        
multi_description = multi_description.index.values.tolist()
multi_description

[]

## Two mode

In [7]:
rows = df['Document Title'].unique()
cols = df['Article'].unique()
two_mode = pd.DataFrame(data=np.zeros((rows.shape[0],cols.shape[0]),dtype=int),index=rows,columns=cols)

row_dict = dict(zip(rows,range(rows.shape[0])))
col_dict = dict(zip(cols,range(cols.shape[0])))

In [8]:
for ind in df.index:
    article_name = df['Document Title'][ind]
    doc_name = df['Article'][ind]
    two_mode.at[article_name, doc_name] += 1
    
two_mode.head()

,90DKASQ,96KMCDKSN,10MTDKAMJ,88DKSRNSPQ,88DKSN,87DKSN,81DRDDMKLWPJAP,01PDDKJMS,85DKLWPASQ,00TRDBDKSMJ,...,10KZRPDKJAMIA,11PPRAKADKHSR,13PPAPMWDKBMCHSR,15TCSBAEDKOS,12MEDKSN,12PPMWSWDKPEC,09DKS,77LLDKHO,12PPMWKADKAJMQ,03DDPHFDKJMS
Structural Logic of Intraorganizational Networks,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bridging firm-internal boundaries for innovation: Directed communication orientation and brokering roles,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Uncovering the relationship between OSS user support networks and OSS popularity,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The firm's knowledge network and the transfer of advice among corporate inventors-A multilevel network study,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Antecedents of creative decision making in organizational crisis: A team-based simulation,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Replace the column names to the new names

In [9]:
new_cols = [article_dict[i] for i in cols]    # change the list of column names to the new version
two_mode.columns = new_cols  # replace the column names
two_mode

,90Polit,96Incon,10Activ,88Crises,88QAP2,87CSS,81FuncTO,01PreTran,85FrLeave,00GovStr,...,10HlthRec,11EMFam1,13TeamCom,15Psych,12GrphTh2,12TeamCon,09RvJack,77Povrty,12EMEvnt,03Equil
Structural Logic of Intraorganizational Networks,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bridging firm-internal boundaries for innovation: Directed communication orientation and brokering roles,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Uncovering the relationship between OSS user support networks and OSS popularity,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The firm's knowledge network and the transfer of advice among corporate inventors-A multilevel network study,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Antecedents of creative decision making in organizational crisis: A team-based simulation,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Knowledge Sharing in a Third-Party-Governed Health and Human Services Network,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Maker-buyer strategic alliances: an integrated framework,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
The impact of deep vertical supply chain relationships upon focal-firm innovation performance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Overcoming Resistance to Organizational Change: Strong Ties and Affective Cooptation,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## One mode

In [10]:
#using dot product of two-mode to get the number of co-citation

A = np.array(two_mode)
G = np.dot(A.T, A)
one_mode = pd.DataFrame(data=G, index=new_cols, columns=new_cols)
    
one_mode.head()

,90Polit,96Incon,10Activ,88Crises,88QAP2,87CSS,81FuncTO,01PreTran,85FrLeave,00GovStr,...,10HlthRec,11EMFam1,13TeamCom,15Psych,12GrphTh2,12TeamCon,09RvJack,77Povrty,12EMEvnt,03Equil
90Polit,89,0,0,2,1,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96Incon,0,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10Activ,0,0,32,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
88Crises,2,0,0,67,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
88QAP2,1,0,0,0,65,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Output the data in csv format

In [11]:
two_mode.to_csv("two_mode.csv")
one_mode.to_csv("one_mode.csv")

# Create Data Script

In [12]:
#Change one-mode dataframe into source-target list

src_dst = pd.DataFrame(columns=['source', 'target', 'value'])

for i in range(len(one_mode)):
    for j in range(i + 1, len(one_mode)):
        value = one_mode.iloc[i, j]
        if(value>0):
            src_dst = src_dst.append([{'source': one_mode.columns.values[i], 
                                       'target': one_mode.columns.values[j],
                                       'value': one_mode.iloc[i, j]}], ignore_index=True, sort=False)
src_dst[: 5]

,source,target,value
0,90Polit,88Crises,2
1,90Polit,88QAP2,1
2,90Polit,87CSS,3
3,90Polit,93InfNet,1
4,90Polit,92Philos,1


## Link List

In [13]:
#Create link list

links_list = []
not_isolates = set()

from scipy import stats



indexes = pd.Index(src_dst['source']
                   .append(src_dst['target'])
                   .reset_index(drop=True).unique())

'''
if(np.std(src_dst["value"]) == 0):
    src_dst["test"] = 0.5
else:
    src_dst["test"] = stats.zscore(src_dst["value"])
    
mins = np.min(src_dst["test"])
maxs = np.max(src_dst["test"])

'''

sigmoid = []
for index, link in src_dst.iterrows():
    sigmoid.append( 1 / (1 + math.exp(-link['value'])) )
    
src_dst["sigmoid"] = sigmoid
mins = np.min(src_dst["sigmoid"])
maxs = np.max(src_dst["sigmoid"])
if(maxs == mins):
    for index, link in src_dst.iterrows():
        not_isolates.add(link['source'])
        not_isolates.add(link['target'])
        record = {"source": link['source'],
                  "target": link['target'],
                  "scaled_cocitation": 0.1,
                  "num_cocitation": link['value']}
        links_list.append(record)

else:
    for index, link in src_dst.iterrows():
        not_isolates.add(link['source'])
        not_isolates.add(link['target'])
        record = {"source": link['source'],
                  "target": link['target'],
                  "scaled_cocitation": 0.05 + (link["sigmoid"] - mins) / ((maxs - mins) / 0.05),
                  "num_cocitation": link['value']}
        links_list.append(record)
    '''
    # print data script
    print('{ target: "' + link['target'] + '", source: "' 
          + link['source'] + '", strength: ' + str(1 / (1 + math.exp(-link['value']))  / 10) 
          +  " ,original: " + str(link['value']) + "},")       
    '''
links_list[: 5]

[{'source': '90Polit',
  'target': '88Crises',
  'scaled_cocitation': 0.07983370986954559,
  'num_cocitation': 2},
 {'source': '90Polit',
  'target': '88QAP2',
  'scaled_cocitation': 0.05,
  'num_cocitation': 1},
 {'source': '90Polit',
  'target': '87CSS',
  'scaled_cocitation': 0.09413447860869012,
  'num_cocitation': 3},
 {'source': '90Polit',
  'target': '93InfNet',
  'scaled_cocitation': 0.05,
  'num_cocitation': 1},
 {'source': '90Polit',
  'target': '92Philos',
  'scaled_cocitation': 0.05,
  'num_cocitation': 1}]

In [14]:
import matplotlib.pyplot as plt

plt.hist(sigmoid)
plt.show()

<Figure size 640x480 with 1 Axes>

In [15]:
nodes_list = []

#article_description = df[["Article", "Description"]].groupby('Article').nth(0).reset_index()

for i in range(len(one_mode)):
    name = one_mode.columns.values.tolist()[i]
    if name in not_isolates:
        year = int(one_mode.columns.values.tolist()[i][0: 2])
        year = (1900 + year) if year > 20 else (2000 + year)
        nodes_list.append({"id": name, 
                            "year": year,
                            "total_citation": int(one_mode.iloc[i, i]),
                            "description": descrptn_dict[name]})
        '''
        print('{id: "' + name +'", label: "' + descrptn_dict[name] + '",' + 'level:' + str(year) + ' ,total_citation:'
         + str(int(one_mode.iloc[i, i])) + "}," )
        '''
nodes_list[: 5]

[{'id': '90Polit',
  'year': 1990,
  'total_citation': 89,
  'description': 'Assessing the Political Landscape - Krackhardt, D; Administrative Science Quarterly - June 1992'},
 {'id': '10Activ',
  'year': 2010,
  'total_citation': 32,
  'description': 'Activating Cross-Boundary Knowledge; Tortoriello, Marco; Krackhardt, David; Academy of Management Journal - Feb 2010'},
 {'id': '88Crises',
  'year': 1988,
  'total_citation': 67,
  'description': 'Informal Network and Organizational Crises - Krackhardt, D; Stern, RN; Social Psychology Quarterly - June 1988'},
 {'id': '88QAP2',
  'year': 1988,
  'total_citation': 65,
  'description': 'Predicting with Networks - Krackhardt, D; Social Networks - Dec. 1988'},
 {'id': '87CSS',
  'year': 1987,
  'total_citation': 58,
  'description': 'Cognitive Social-Structures - Krackhardt, D; Social Networks - Jun 1987'}]

## Output to files

In [16]:
json_prep = {"links":links_list, "nodes":nodes_list}
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

In [17]:
filename_out = 'link_node.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()

In [18]:
new_names = pd.read_csv("codeAndNamev2.csv")
new_names.head()

,New article name,Article,Description
0,00GovStr,00TRDBDKSMJ,"Redudant Governance Structures - Rowley, T; Be..."
1,00LatCon,00ELDKQQ,Spreading and shifting costs of lateral contro...
2,00WrkGrp,00PJHKMCDKODHDP,"Choosing Work Group Members; Hinds, PJ; Carley..."
3,01PreTran,01PDDKJMS,Pre-transitive balance mechanisms for signed n...
4,02Festsch,02NEFDKSN,Festschrift for Linton C Freeman: Introduction...


In [19]:
newInfo = list(zip(new_names['New article name'],new_names['Description']))
oldID = list(new_names['Article'])

# dictionary that look up the old article id for the new id and description
nodeDict = dict(zip(oldID,newInfo))
nodeDict

{'00TRDBDKSMJ': ('00GovStr',
  'Redudant Governance Structures - Rowley, T; Behrens, D; Krackhardt, D; Strategic Management Journal - March 2000'),
 '00ELDKQQ': ('00LatCon',
  'Spreading and shifting costs of lateral control among peers - Lazega, E; Krackhardt, D - Quality & Quantity - May 2000'),
 '00PJHKMCDKODHDP': ('00WrkGrp',
  'Choosing Work Group Members; Hinds, PJ; Carley, KM; Krackhardt, D; et al.; Organizational Behavior and Human Decision Processes - Mar 2000'),
 '01PDDKJMS': ('01PreTran',
  'Pre-transitive balance mechanisms for signed networks - Doreian, P; Krackhardt, D - Journal of Mathematical Sociology - 2001'),
 '02NEFDKSN': ('02Festsch',
  'Festschrift for Linton C Freeman: Introduction - Friedkin, NE; Krackhardt, D - Social Networks - Oct 2002'),
 '02DKMKSN': ('02Simmel3',
  'Structure, culture, and Simmelian ties in entrepreneurial firms - Krackhardt, D; Kilduff, M - Social Networks - July 2002'),
 '03DDPHFDKJMS': ('03Equil',
  'An equilibrium-correction model for d

In [20]:
list(zip(new_names['New article name'],new_names['Description']))[: 5]

[('00GovStr',
  'Redudant Governance Structures - Rowley, T; Behrens, D; Krackhardt, D; Strategic Management Journal - March 2000'),
 ('00LatCon',
  'Spreading and shifting costs of lateral control among peers - Lazega, E; Krackhardt, D - Quality & Quantity - May 2000'),
 ('00WrkGrp',
  'Choosing Work Group Members; Hinds, PJ; Carley, KM; Krackhardt, D; et al.; Organizational Behavior and Human Decision Processes - Mar 2000'),
 ('01PreTran',
  'Pre-transitive balance mechanisms for signed networks - Doreian, P; Krackhardt, D - Journal of Mathematical Sociology - 2001'),
 ('02Festsch',
  'Festschrift for Linton C Freeman: Introduction - Friedkin, NE; Krackhardt, D - Social Networks - Oct 2002')]